In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pythainlp import word_tokenize
from pythainlp.ulmfit import process_thai

In [10]:
with open("wisesight-sentiment-master/kaggle-competition/train.txt", encoding="utf8") as f:
    texts = [line.strip() for line in f.readlines()]

with open("wisesight-sentiment-master/kaggle-competition/train_label.txt", encoding="utf8") as f:
    categories = [line.strip() for line in f.readlines()]

all_df = pd.DataFrame({"category":categories, "texts":texts})
all_df.to_csv('all_df.csv',index=False)
all_df.shape

(24063, 2)

In [11]:
df = pd.read_excel("1000CommendfromYoutube.xlsx")
df.head()

,Unnamed: 0,Unnamed: 0.1,dateTime,authorName,Message,processed,wc,uwc,preds,time2,NewCol,Gender
0,0,0,2023-01-09 06:01:26,คนไทย Australian,1,1,1,1,neu,2023-01-09 06:01:26,0,Undefine
1,1,1,2023-01-09 06:01:27,อากู๋ channel,1,1,1,1,neu,2023-01-09 06:01:26,0,Undefine
2,2,2,2023-01-09 06:01:31,Final,ๆ,ๆ,1,1,pos,2023-01-09 06:01:26,0,Undefine
3,3,3,2023-01-09 06:01:32,Pornchai S.,hi,hi,1,1,neu,2023-01-09 06:01:26,0,Undefine
4,4,4,2023-01-09 06:01:34,Bao Tsunami,:face-red-heart-shape::face-red-heart-shape::f...,:|face-red-heart-shape|::|face-red-heart-shape...,7,3,neu,2023-01-09 06:01:26,0,Undefine


In [12]:
df_bar = pd.DataFrame({'count' : df.groupby( ["preds"] ).size()}).reset_index()
df_graphbar = pd.DataFrame({'count' : df.groupby( ['NewCol','preds'] ).size()}).reset_index()
df_graphgender = pd.DataFrame({'count' : df.groupby(['Gender']).size()}).reset_index()
df_line = df[['NewCol', 'Gender']].pivot_table(index="NewCol", columns="Gender",  aggfunc=len).fillna(0).astype(int).reset_index()
df_line['Total'] = df_line['Female'] + df_line['Male'] + df_line['Undefine']
df_graphpie = pd.DataFrame({'count' : df.groupby( ['preds'] ).size()}).reset_index()
df_people = pd.DataFrame({'count' : df.groupby(['authorName']).size()})
df_graphpeople = df_people['count'].sort_values(ascending=False).head(10).reset_index()

In [13]:
from dash import Dash, dcc, html, Input, Output, ctx
from jupyter_dash import JupyterDash
import plotly.express as px
import pandas as pd

sentiment = ['pos', 'neg', 'neu']

app = JupyterDash(__name__)

fig = px.line(df_line, x="NewCol", y="Total", text="Total")
fig.update_traces(textposition='top center')

fig2 = px.bar(df_graphbar, x="preds", y="count",color="preds",color_discrete_sequence=["red", "gray","green"])

fig3 = px.bar(df_graphgender, x="Gender", y="count", color="Gender")

fig4 = px.pie(df_graphpie, values="count", names="preds", hole=.3)

fig5 = px.bar(df_graphpeople, x="authorName", y="count", text="count")

app.layout = html.Div([
    
    html.Div(children=[
        html.H1(children='Total #Commend by time'),

        html.Div(children='''
               The sentiment of sentense.
        '''),
        
        dcc.Checklist(
                  id = "checklist1",
                  options=[{'label': i, 'value': i} for i in sentiment],
                  value=['pos', 'neg', 'neu'], #Initial values,
                  inline=True),

        dcc.Graph(
            id='example-graph1',
            figure=fig, clear_on_unhover=True, clickData=None
        ),
        
        html.H1(children='%Contribution of Pos-Neu-Neg sentiment'),
        
        dcc.Graph(
            id='example-graphpie4',
            figure=fig4
        ),
        
        dcc.RangeSlider(
            min(df['NewCol']),
            max(df['NewCol'])+1,
            1,
            value=[min(df['NewCol']), max(df['NewCol'])],
            id='range-slider')
    ],style={'padding': 10, 'flex': 1}),
    
    html.Div(children=[
        html.H1(children='Total #Commend by Gender'),
        html.Div(children='''
            Click to clear filter.
        '''),
        html.Button('Clear Filter', id='btn1', n_clicks=0),
        
        dcc.Graph(
            id='example-graph',
            figure=fig3, clickData=None
        ),
        
        html.H1(children='Top #Commend by Authorname'),
        
        dcc.Graph(
            id='example-graphpeople',
            figure=fig5
        )

    ],style={'padding': 10, 'flex': 1})
    ,
],style={'display': 'flex', 'flexDirection': 'row', 'flex-wrap': 'wrap'})

@app.callback(
    Output('example-graph1', 'figure'),
    Input('checklist1', 'value'))

def update_fig1(x):
    n_df_line = df[(df['preds'].isin(x))][['NewCol','preds']].groupby('NewCol').size().reset_index()
    n_df_line.columns = ['Time', 'Total']
    fig = px.line(n_df_line, x="Time", y="Total",text="Total")
    fig.update_traces(textposition='top center')
    #n_df_graphgender = df[(df['preds'].isin(x))][['Gender','preds']].groupby('Gender').size().reset_index()
    #n_df_graphgender.columns = ['Gender', 'Total']
    #fig3 = px.bar(n_df_graphgender, x="Gender", y="Total", color="Gender")
    return fig

@app.callback(
    Output('example-graph', 'figure'),
    Input('checklist1', 'value'),
    Input('example-graph1','hoverData'),
    Input('example-graph1', 'clickData'),
    Input('btn1', 'n_clicks'))

def update_fig2(x, hover_data, click_data, n_click):
    if ctx.triggered_id == "btn1":
        n_df_graphgender = df[(df['preds'].isin(x))][['Gender','preds']].groupby('Gender').size().reset_index()
        n_df_graphgender.columns = ['Gender', 'Total']
        fig3 = px.bar(n_df_graphgender, x="Gender", y="Total", color="Gender")
        return fig3
    if hover_data is None:
        if click_data is None:
            n_df_graphgender = df[(df['preds'].isin(x))][['Gender','preds']].groupby('Gender').size().reset_index()
            n_df_graphgender.columns = ['Gender', 'Total']
            fig3 = px.bar(n_df_graphgender, x="Gender", y="Total", color="Gender")
            return fig3
        else:
            Specific_time = click_data['points'][0]['x']
            n_df_graphgender = df[(df['preds'].isin(x)) & (df['NewCol'] == Specific_time)][['Gender','preds']].groupby('Gender').size().reset_index()
            n_df_graphgender.columns = ['Gender', 'Total']
            fig3 = px.bar(n_df_graphgender, x="Gender", y="Total", color="Gender")
            return fig3
    else:
        Specific_time = hover_data['points'][0]['x']
        n_df_graphgender = df[(df['preds'].isin(x)) & (df['NewCol'] == Specific_time)][['Gender','preds']].groupby('Gender').size().reset_index()
        n_df_graphgender.columns = ['Gender', 'Total']
        fig3 = px.bar(n_df_graphgender, x="Gender", y="Total", color="Gender")
        return fig3

@app.callback(
    Output('example-graphpie4', 'figure'),
    Output('example-graphpeople', 'figure'),
    Input('range-slider', 'value'),
    Input('checklist1', 'value'))

def updatefig3(range_data, x):
    min_time = range_data[0]
    max_time = range_data[1]
    n_df_pie = df[(df['NewCol'] >= min_time) & (df['NewCol'] <= max_time)][['preds']].groupby('preds').size().reset_index()
    n_df_pie.columns = ['preds', 'count']
    fig4 = px.pie(n_df_pie, values="count", names="preds", hole=.3)
    n_df_people = pd.DataFrame({'count' : df[(df['NewCol'] >= min_time) & (df['NewCol'] <= max_time) & df['preds'].isin(x)].groupby(['authorName']).size()})
    n_df_graphpeople = n_df_people['count'].sort_values(ascending=False).head(10).reset_index()
    fig5 = px.bar(n_df_graphpeople, x="authorName", y="count", text="count")
    return fig4, fig5

@app.callback(
    Output('example-graph1', 'clickData'),
    Input('btn1', 'n_clicks'))
def clear_clickdata(x):
    return None


if __name__ == '__main__':
    app.run_server(debug=True, port=8054)

Dash app running on http://127.0.0.1:8054/
